# Books

Tra le varie colonne della tabella sono risultate significative le seguenti:

- iillllll (Titolo libro)
- iiliriiriliiriliiiirlrllllilii (prezzo)
- iiliriiriliiriliiiiririri (data pubblicazione)
- iiliriiriliiriliiiirirrirri (numero pagine)
- iiliriiriliiriliiiirlrrl (recensione)

In [1]:
#Questo parametro indica il numero di valori che devo analizzare per stabilire l'etichetta corretta.
RECORDS_NUMBER= 100

# LIBRERIE

In [3]:
import requests
from requests.exceptions import ConnectionError
from transformers import pipeline

from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# Labels candidate

In [215]:
#Qui scelgo delle label generiche che potrebbero essere assocciate alle varie colonne della tabella.
labels=['Book title', 'Price', 'Pages', 'Publish Date', 'Description']

# Label : Book title

In [24]:
url='http://localhost:8123/?query=SELECT top 100 iillllll FROM extracted_data.57632537416d4cd19f3965026ab51caf'

try:
    r = requests.get(url)
except ConnectionError:
        print("Il collegamento al database non è attivo.")

        
#L'utf-8-sig è una variante Python di UTF-8 che ci permette di eliminare, se presenti, eventuali carattere UTF-8 BOM.
values= r.content.decode('utf-8-sig')


#Controllo i parametri inseriti dall'utente
if (values[6:8]=='60'):
    print("ATTENZIONE! La tabella indicata non esiste.")
elif (values[6:8]=='47'):
    print("ATTENZIONE! La colonna indicata non esiste.")
else:
    print("I dati sono stati caricati correttamente.")

I dati sono stati caricati correttamente.


In [25]:
values

'Sarah\\\'s Key : Tatiana De Rosnay : 9780312370848\n\ufeffUnbearable Lightness: A Story of Loss and Gain : Portia De Rossi : 9781439177785\n\ufeffThe Slap: One day, at a suburban barbeque, a man slaps a child who is not his own... : Tsiolkas Christos : 9781848877993\n\ufeffPublication Manual of the American Psychological Association (Publication Manual of the American Psychological Association) : American Psychological Association : 9781433805615\n\ufeffJasper Jones : Craig Silvey : 9780099537540\n\ufeffCrazy Love: Overwhelmed by a Relentless God : Francis Chan, Danae Yankoski : 9781434768513\n\ufeffUnderstanding Exposure: How to Shoot Great Photographs with Any Camera : Bryan Peterson : 9780817439392\n\ufeffThe Help : Kathryn Stockett : 9780141047706\n\ufeffThe 4-hour Work Week: Escape the 9-5, Live Anywhere and Join the New Rich : Timothy Ferriss : 9780091929114\n\ufeffThe Playbook: Suit Up. Score Chicks. Be Awesome. : Neil Patrick Harris, Matt Kuhn : 9781439196830\n\ufeffInvincible

In [26]:
# PREPROCESSING

In [27]:
values = values.replace("\ufeff", "")
values = values.replace("\\", "")

In [28]:
values

'Sarah\'s Key : Tatiana De Rosnay : 9780312370848\nUnbearable Lightness: A Story of Loss and Gain : Portia De Rossi : 9781439177785\nThe Slap: One day, at a suburban barbeque, a man slaps a child who is not his own... : Tsiolkas Christos : 9781848877993\nPublication Manual of the American Psychological Association (Publication Manual of the American Psychological Association) : American Psychological Association : 9781433805615\nJasper Jones : Craig Silvey : 9780099537540\nCrazy Love: Overwhelmed by a Relentless God : Francis Chan, Danae Yankoski : 9781434768513\nUnderstanding Exposure: How to Shoot Great Photographs with Any Camera : Bryan Peterson : 9780817439392\nThe Help : Kathryn Stockett : 9780141047706\nThe 4-hour Work Week: Escape the 9-5, Live Anywhere and Join the New Rich : Timothy Ferriss : 9780091929114\nThe Playbook: Suit Up. Score Chicks. Be Awesome. : Neil Patrick Harris, Matt Kuhn : 9781439196830\nInvincible Ultimate Collection: v. 6 : Ryan Ottley, Robert Kirkman : 978

In [29]:
values = values.split("\n")

In [30]:
values.pop()

''

In [32]:
values

["Sarah's Key : Tatiana De Rosnay : 9780312370848",
 'Unbearable Lightness: A Story of Loss and Gain : Portia De Rossi : 9781439177785',
 'The Slap: One day, at a suburban barbeque, a man slaps a child who is not his own... : Tsiolkas Christos : 9781848877993',
 'Publication Manual of the American Psychological Association (Publication Manual of the American Psychological Association) : American Psychological Association : 9781433805615',
 'Jasper Jones : Craig Silvey : 9780099537540',
 'Crazy Love: Overwhelmed by a Relentless God : Francis Chan, Danae Yankoski : 9781434768513',
 'Understanding Exposure: How to Shoot Great Photographs with Any Camera : Bryan Peterson : 9780817439392',
 'The Help : Kathryn Stockett : 9780141047706',
 'The 4-hour Work Week: Escape the 9-5, Live Anywhere and Join the New Rich : Timothy Ferriss : 9780091929114',
 'The Playbook: Suit Up. Score Chicks. Be Awesome. : Neil Patrick Harris, Matt Kuhn : 9781439196830',
 'Invincible Ultimate Collection: v. 6 : Rya

In [33]:
# PREVISIONE LABEL

In [34]:
#Struttura: dictionary[label] -> [89%, 93%, 97%, 32%, 99% , ...]
dictionary={}

#Sruttura ausiliaria: permette di stabilire il numero di volte in cui ciascuna etichetta è stata predetta
occorrenze={}


#Crea una lista vuota per ciascuna label 
for label in labels:
    dictionary[label]=[]
    occorrenze[label]=0
    

classifier = pipeline("zero-shot-classification")
    
labels_number= len(labels)

#Vengono effettuate le previsioni attraverso il modello.
#Le previsioni vengono memorizzate nel dizionario.
for i in range(RECORDS_NUMBER):
    text = values[i]
    result=classifier(text, labels)
    occorrenze[result['labels'][0]]+=1
    for y in range(labels_number):
        label= result['labels'][y]
        score= result['scores'][y]
        dictionary[label].append(score)    
        
        
risultati={}
for label in labels:
    risultati[label]= sum(dictionary[label])/RECORDS_NUMBER

max_score= 0

for label, score in risultati.items():  
    if score > max_score:
        label_predict=label
        max_score=score

print("Etichetta predetta: %s \n\n\n" % label_predict)
print("Grado di affidabilità: %.3f\n" % max_score)
print("%d/%d -> Questa etichetta è stata predetta %d volte su %d." 
      % (occorrenze[label_predict], RECORDS_NUMBER, occorrenze[label_predict], RECORDS_NUMBER))

Etichetta predetta: Book title 



Grado di affidabilità: 0.513

78/100 -> Questa etichetta è stata predetta 78 volte su 100.


# Label : Price

In [36]:
url='http://localhost:8123/?query=SELECT top 100 iiliriiriliiriliiiirlrllllilii  FROM extracted_data.57632537416d4cd19f3965026ab51caf'

try:
    r = requests.get(url)
except ConnectionError:
        print("Il collegamento al database non è attivo.")

        
#L'utf-8-sig è una variante Python di UTF-8 che ci permette di eliminare, se presenti, eventuali carattere UTF-8 BOM.
values= r.content.decode('utf-8-sig')


#Controllo i parametri inseriti dall'utente
if (values[6:8]=='60'):
    print("ATTENZIONE! La tabella indicata non esiste.")
elif (values[6:8]=='47'):
    print("ATTENZIONE! La colonna indicata non esiste.")
else:
    print("I dati sono stati caricati correttamente.")

I dati sono stati caricati correttamente.


In [37]:
values

'$5.99\n$13.49\n\n$22.49\n\n$6.99\n$15.99\n\n\n$6.49\n$26.24\n$15.93\n$12.99\n$9.49\n\n\n$10.49\n$15.32\n$4.45\n$7.99\n$6.66\n$32.79\n$5.99\n$17.09\n$12.99\n$8.99\n$5.99\n\n$2.62\n\n$6.49\n$6.99\n$15.99\n\n$9.98\n\n$10.63\n$7.45\n$6.49\n$16.33\n$16.99\n$11.11\n$6.02\n$7.49\n$6.99\n$12.68\n$24.64\n$8.73\n\n$8.99\n$16.49\n$5.99\n$6.49\n$38.49\n\n$15.49\n$17.22\n\n\n$5.43\n\n$12.24\n$5.99\n$7.49\n$7.99\n$8.99\n$13.49\n$6.99\n$19.12\n$28.45\n$3.38\n$18.61\n$9.03\n$7.99\n$12.49\n$49.86\n$8.99\n$9.49\n$7.50\n$13.55\n$18.69\n$15.99\n$6.99\n$13.99\n$18.49\n$14.99\n$7.99\n\n$9.99\n$6.99\n$8.10\n$12.99\n$5.99\n\n$9.11\n$11.61\n$10.28\n$34.30\n$8.99\n$8.71\n'

In [38]:
values = values.split("\n")

In [39]:
values

['$5.99',
 '$13.49',
 '',
 '$22.49',
 '',
 '$6.99',
 '$15.99',
 '',
 '',
 '$6.49',
 '$26.24',
 '$15.93',
 '$12.99',
 '$9.49',
 '',
 '',
 '$10.49',
 '$15.32',
 '$4.45',
 '$7.99',
 '$6.66',
 '$32.79',
 '$5.99',
 '$17.09',
 '$12.99',
 '$8.99',
 '$5.99',
 '',
 '$2.62',
 '',
 '$6.49',
 '$6.99',
 '$15.99',
 '',
 '$9.98',
 '',
 '$10.63',
 '$7.45',
 '$6.49',
 '$16.33',
 '$16.99',
 '$11.11',
 '$6.02',
 '$7.49',
 '$6.99',
 '$12.68',
 '$24.64',
 '$8.73',
 '',
 '$8.99',
 '$16.49',
 '$5.99',
 '$6.49',
 '$38.49',
 '',
 '$15.49',
 '$17.22',
 '',
 '',
 '$5.43',
 '',
 '$12.24',
 '$5.99',
 '$7.49',
 '$7.99',
 '$8.99',
 '$13.49',
 '$6.99',
 '$19.12',
 '$28.45',
 '$3.38',
 '$18.61',
 '$9.03',
 '$7.99',
 '$12.49',
 '$49.86',
 '$8.99',
 '$9.49',
 '$7.50',
 '$13.55',
 '$18.69',
 '$15.99',
 '$6.99',
 '$13.99',
 '$18.49',
 '$14.99',
 '$7.99',
 '',
 '$9.99',
 '$6.99',
 '$8.10',
 '$12.99',
 '$5.99',
 '',
 '$9.11',
 '$11.61',
 '$10.28',
 '$34.30',
 '$8.99',
 '$8.71',
 '']

In [40]:
values = list(filter(lambda x : x != '', values))

In [41]:
values

['$5.99',
 '$13.49',
 '$22.49',
 '$6.99',
 '$15.99',
 '$6.49',
 '$26.24',
 '$15.93',
 '$12.99',
 '$9.49',
 '$10.49',
 '$15.32',
 '$4.45',
 '$7.99',
 '$6.66',
 '$32.79',
 '$5.99',
 '$17.09',
 '$12.99',
 '$8.99',
 '$5.99',
 '$2.62',
 '$6.49',
 '$6.99',
 '$15.99',
 '$9.98',
 '$10.63',
 '$7.45',
 '$6.49',
 '$16.33',
 '$16.99',
 '$11.11',
 '$6.02',
 '$7.49',
 '$6.99',
 '$12.68',
 '$24.64',
 '$8.73',
 '$8.99',
 '$16.49',
 '$5.99',
 '$6.49',
 '$38.49',
 '$15.49',
 '$17.22',
 '$5.43',
 '$12.24',
 '$5.99',
 '$7.49',
 '$7.99',
 '$8.99',
 '$13.49',
 '$6.99',
 '$19.12',
 '$28.45',
 '$3.38',
 '$18.61',
 '$9.03',
 '$7.99',
 '$12.49',
 '$49.86',
 '$8.99',
 '$9.49',
 '$7.50',
 '$13.55',
 '$18.69',
 '$15.99',
 '$6.99',
 '$13.99',
 '$18.49',
 '$14.99',
 '$7.99',
 '$9.99',
 '$6.99',
 '$8.10',
 '$12.99',
 '$5.99',
 '$9.11',
 '$11.61',
 '$10.28',
 '$34.30',
 '$8.99',
 '$8.71']

In [42]:
newRECORDS_NUMBER = len(values)
len(values)

83

In [43]:
#Struttura: dictionary[label] -> [89%, 93%, 97%, 32%, 99% , ...]
dictionary={}

#Sruttura ausiliaria: permette di stabilire il numero di volte in cui ciascuna etichetta è stata predetta
occorrenze={}


#Crea una lista vuota per ciascuna label 
for label in labels:
    dictionary[label]=[]
    occorrenze[label]=0
    

classifier = pipeline("zero-shot-classification")
    
labels_number= len(labels)

#Vengono effettuate le previsioni attraverso il modello.
#Le previsioni vengono memorizzate nel dizionario.
for i in range(newRECORDS_NUMBER):
    text = values[i]
    result=classifier(text, labels)
    occorrenze[result['labels'][0]]+=1
    for y in range(labels_number):
        label= result['labels'][y]
        score= result['scores'][y]
        dictionary[label].append(score)    
        
        
risultati={}
for label in labels:
    risultati[label]= sum(dictionary[label])/newRECORDS_NUMBER

max_score= 0

for label, score in risultati.items():  
    if score > max_score:
        label_predict=label
        max_score=score

print("Etichetta predetta: %s \n\n\n" % label_predict)
print("Grado di affidabilità: %.3f\n" % max_score)
print("%d/%d -> Questa etichetta è stata predetta %d volte su %d." 
      % (occorrenze[label_predict], newRECORDS_NUMBER, occorrenze[label_predict], newRECORDS_NUMBER))

Etichetta predetta: Price 



Grado di affidabilità: 0.733

83/83 -> Questa etichetta è stata predetta 83 volte su 83.


# Label : Publish Date

In [73]:
url='http://localhost:8123/?query=SELECT top 100 iiliriiriliiriliiiiririri  FROM extracted_data.57632537416d4cd19f3965026ab51caf'

try:
    r = requests.get(url)
except ConnectionError:
        print("Il collegamento al database non è attivo.")

        
#L'utf-8-sig è una variante Python di UTF-8 che ci permette di eliminare, se presenti, eventuali carattere UTF-8 BOM.
values= r.content.decode('utf-8-sig')


#Controllo i parametri inseriti dall'utente
if (values[6:8]=='60'):
    print("ATTENZIONE! La tabella indicata non esiste.")
elif (values[6:8]=='47'):
    print("ATTENZIONE! La colonna indicata non esiste.")
else:
    print("I dati sono stati caricati correttamente.")

I dati sono stati caricati correttamente.


In [74]:
values

'30 September 2008\n01 November 2010\n\n15 July 2009\n29 April 2010\n10 March 2009\n05 August 2010\n04 May 2010\n06 January 2011\n01 November 2010\n08 March 2011\n01 April 2009\n\n06 April 2010\n28 September 2010\n01 November 2007\n07 March 2010\n25 June 2001\n27 June 2003\n25 March 2011\n22 September 1997\n28 September 2010\n01 February 2011\n27 April 2010\n16 January 2004\n31 October 1999\n02 March 2006\n05 January 2009\n28 April 1994\n02 September 2010\n03 August 2010\n28 July 2004\n01 July 2003\n27 April 2010\n25 June 1992\n03 March 2011\n01 February 2007\n01 January 2010\n26 October 2010\n02 September 2010\n02 November 2010\n14 May 2010\n21 November 2002\n01 April 2007\n01 February 2009\n04 June 2010\n21 June 2010\n20 October 1999\n03 June 2010\n03 November 2009\n03 December 2009\n01 October 1988\n\n01 December 2009\n19 March 2009\n23 August 2006\n12 November 2007\n05 March 2009\n01 July 1999\n01 September 2000\n25 February 2010\n01 January 2010\n25 January 2011\n04 January 2011\n

In [75]:
values = values.split("\n")
values

['30 September 2008',
 '01 November 2010',
 '',
 '15 July 2009',
 '29 April 2010',
 '10 March 2009',
 '05 August 2010',
 '04 May 2010',
 '06 January 2011',
 '01 November 2010',
 '08 March 2011',
 '01 April 2009',
 '',
 '06 April 2010',
 '28 September 2010',
 '01 November 2007',
 '07 March 2010',
 '25 June 2001',
 '27 June 2003',
 '25 March 2011',
 '22 September 1997',
 '28 September 2010',
 '01 February 2011',
 '27 April 2010',
 '16 January 2004',
 '31 October 1999',
 '02 March 2006',
 '05 January 2009',
 '28 April 1994',
 '02 September 2010',
 '03 August 2010',
 '28 July 2004',
 '01 July 2003',
 '27 April 2010',
 '25 June 1992',
 '03 March 2011',
 '01 February 2007',
 '01 January 2010',
 '26 October 2010',
 '02 September 2010',
 '02 November 2010',
 '14 May 2010',
 '21 November 2002',
 '01 April 2007',
 '01 February 2009',
 '04 June 2010',
 '21 June 2010',
 '20 October 1999',
 '03 June 2010',
 '03 November 2009',
 '03 December 2009',
 '01 October 1988',
 '',
 '01 December 2009',
 '19 

In [76]:
values = list(filter(lambda x : x != '', values))

In [77]:
values

['30 September 2008',
 '01 November 2010',
 '15 July 2009',
 '29 April 2010',
 '10 March 2009',
 '05 August 2010',
 '04 May 2010',
 '06 January 2011',
 '01 November 2010',
 '08 March 2011',
 '01 April 2009',
 '06 April 2010',
 '28 September 2010',
 '01 November 2007',
 '07 March 2010',
 '25 June 2001',
 '27 June 2003',
 '25 March 2011',
 '22 September 1997',
 '28 September 2010',
 '01 February 2011',
 '27 April 2010',
 '16 January 2004',
 '31 October 1999',
 '02 March 2006',
 '05 January 2009',
 '28 April 1994',
 '02 September 2010',
 '03 August 2010',
 '28 July 2004',
 '01 July 2003',
 '27 April 2010',
 '25 June 1992',
 '03 March 2011',
 '01 February 2007',
 '01 January 2010',
 '26 October 2010',
 '02 September 2010',
 '02 November 2010',
 '14 May 2010',
 '21 November 2002',
 '01 April 2007',
 '01 February 2009',
 '04 June 2010',
 '21 June 2010',
 '20 October 1999',
 '03 June 2010',
 '03 November 2009',
 '03 December 2009',
 '01 October 1988',
 '01 December 2009',
 '19 March 2009',
 '

In [78]:
newRECORDS_NUMBER= len(values)
len(values)

96

In [79]:
#Struttura: dictionary[label] -> [89%, 93%, 97%, 32%, 99% , ...]
dictionary={}

#Sruttura ausiliaria: permette di stabilire il numero di volte in cui ciascuna etichetta è stata predetta
occorrenze={}


#Crea una lista vuota per ciascuna label 
for label in labels:
    dictionary[label]=[]
    occorrenze[label]=0
    

classifier = pipeline("zero-shot-classification")
    
labels_number= len(labels)

#Vengono effettuate le previsioni attraverso il modello.
#Le previsioni vengono memorizzate nel dizionario.
for i in range(newRECORDS_NUMBER):
    text = values[i]
    result=classifier(text, labels)
    occorrenze[result['labels'][0]]+=1
    for y in range(labels_number):
        label= result['labels'][y]
        score= result['scores'][y]
        dictionary[label].append(score)    
        
        
risultati={}
for label in labels:
    risultati[label]= sum(dictionary[label])/newRECORDS_NUMBER

max_score= 0

for label, score in risultati.items():  
    if score > max_score:
        label_predict=label
        max_score=score

print("Etichetta predetta: %s \n\n\n" % label_predict)
print("Grado di affidabilità: %.3f\n" % max_score)
print("%d/%d -> Questa etichetta è stata predetta %d volte su %d." 
      % (occorrenze[label_predict], newRECORDS_NUMBER, occorrenze[label_predict], newRECORDS_NUMBER))

Etichetta predetta: Publish Date 



Grado di affidabilità: 0.462

96/96 -> Questa etichetta è stata predetta 96 volte su 96.


# Label : Pages

In [82]:
url='http://localhost:8123/?query=SELECT top 100 iiliriiriliiriliiiirirrirri FROM extracted_data.57632537416d4cd19f3965026ab51caf'

try:
    r = requests.get(url)
except ConnectionError:
        print("Il collegamento al database non è attivo.")

        
#L'utf-8-sig è una variante Python di UTF-8 che ci permette di eliminare, se presenti, eventuali carattere UTF-8 BOM.
values= r.content.decode('utf-8-sig')


#Controllo i parametri inseriti dall'utente
if (values[6:8]=='60'):
    print("ATTENZIONE! La tabella indicata non esiste.")
elif (values[6:8]=='47'):
    print("ATTENZIONE! La colonna indicata non esiste.")
else:
    print("I dati sono stati caricati correttamente.")

I dati sono stati caricati correttamente.


In [83]:
values

'293 pages\n308 pages\n\n300 pages\n304 pages\n186 pages\n176 pages\n464 pages\n416 pages\n176 pages\n336 pages\n292 pages\n\n512 pages\n176 pages\n390 pages\n218 pages\n400 pages\n128 pages\n176 pages\n36 pages\n960 pages\n320 pages\n307 pages\n464 pages\n286 pages\n26 pages\n224 pages\n112 pages\n448 pages\n248 pages\n168 pages\n200 pages\n368 pages\n718 pages\n352 pages\n192 pages\n\n413 pages\n\n255 pages\n220 pages\n26 pages\n224 pages\n213 pages\n224 pages\n400 pages\n416 pages\n368 pages\n480 pages\n320 pages\n\n\n\n192 pages\n240 pages\n256 pages\n352 pages\n192 pages\n14 pages\n304 pages\n396 pages\n416 pages\n304 pages\n274 pages\n256 pages\n232 pages\n200 pages\n224 pages\n191 pages\n240 pages\n192 pages\n292 pages\n32 pages\n347 pages\n1088 pages\n224 pages\n328 pages\n\n336 pages\n224 pages\n224 pages\n\n208 pages\n433 pages\n416 pages\n560 pages\n432 pages\n229 pages\n596 pages\n384 pages\n192 pages\n24 pages\n272 pages\n212 pages\n192 pages\n256 pages\n832 pages\n240 pag

In [84]:
values = values.split("\n")

In [85]:
values

['293 pages',
 '308 pages',
 '',
 '300 pages',
 '304 pages',
 '186 pages',
 '176 pages',
 '464 pages',
 '416 pages',
 '176 pages',
 '336 pages',
 '292 pages',
 '',
 '512 pages',
 '176 pages',
 '390 pages',
 '218 pages',
 '400 pages',
 '128 pages',
 '176 pages',
 '36 pages',
 '960 pages',
 '320 pages',
 '307 pages',
 '464 pages',
 '286 pages',
 '26 pages',
 '224 pages',
 '112 pages',
 '448 pages',
 '248 pages',
 '168 pages',
 '200 pages',
 '368 pages',
 '718 pages',
 '352 pages',
 '192 pages',
 '',
 '413 pages',
 '',
 '255 pages',
 '220 pages',
 '26 pages',
 '224 pages',
 '213 pages',
 '224 pages',
 '400 pages',
 '416 pages',
 '368 pages',
 '480 pages',
 '320 pages',
 '',
 '',
 '',
 '192 pages',
 '240 pages',
 '256 pages',
 '352 pages',
 '192 pages',
 '14 pages',
 '304 pages',
 '396 pages',
 '416 pages',
 '304 pages',
 '274 pages',
 '256 pages',
 '232 pages',
 '200 pages',
 '224 pages',
 '191 pages',
 '240 pages',
 '192 pages',
 '292 pages',
 '32 pages',
 '347 pages',
 '1088 pages',
 '2

In [86]:
values = list (filter(lambda x : x != '', values))

In [87]:
values

['293 pages',
 '308 pages',
 '300 pages',
 '304 pages',
 '186 pages',
 '176 pages',
 '464 pages',
 '416 pages',
 '176 pages',
 '336 pages',
 '292 pages',
 '512 pages',
 '176 pages',
 '390 pages',
 '218 pages',
 '400 pages',
 '128 pages',
 '176 pages',
 '36 pages',
 '960 pages',
 '320 pages',
 '307 pages',
 '464 pages',
 '286 pages',
 '26 pages',
 '224 pages',
 '112 pages',
 '448 pages',
 '248 pages',
 '168 pages',
 '200 pages',
 '368 pages',
 '718 pages',
 '352 pages',
 '192 pages',
 '413 pages',
 '255 pages',
 '220 pages',
 '26 pages',
 '224 pages',
 '213 pages',
 '224 pages',
 '400 pages',
 '416 pages',
 '368 pages',
 '480 pages',
 '320 pages',
 '192 pages',
 '240 pages',
 '256 pages',
 '352 pages',
 '192 pages',
 '14 pages',
 '304 pages',
 '396 pages',
 '416 pages',
 '304 pages',
 '274 pages',
 '256 pages',
 '232 pages',
 '200 pages',
 '224 pages',
 '191 pages',
 '240 pages',
 '192 pages',
 '292 pages',
 '32 pages',
 '347 pages',
 '1088 pages',
 '224 pages',
 '328 pages',
 '336 page

In [89]:
newRECORDS_NUMBER = len(values)
len(values)

91

In [90]:
#Struttura: dictionary[label] -> [89%, 93%, 97%, 32%, 99% , ...]
dictionary={}

#Sruttura ausiliaria: permette di stabilire il numero di volte in cui ciascuna etichetta è stata predetta
occorrenze={}


#Crea una lista vuota per ciascuna label 
for label in labels:
    dictionary[label]=[]
    occorrenze[label]=0
    

classifier = pipeline("zero-shot-classification")
    
labels_number= len(labels)

#Vengono effettuate le previsioni attraverso il modello.
#Le previsioni vengono memorizzate nel dizionario.
for i in range(newRECORDS_NUMBER):
    text = values[i]
    result=classifier(text, labels)
    occorrenze[result['labels'][0]]+=1
    for y in range(labels_number):
        label= result['labels'][y]
        score= result['scores'][y]
        dictionary[label].append(score)    
        
        
risultati={}
for label in labels:
    risultati[label]= sum(dictionary[label])/newRECORDS_NUMBER

max_score= 0

for label, score in risultati.items():  
    if score > max_score:
        label_predict=label
        max_score=score

print("Etichetta predetta: %s \n\n\n" % label_predict)
print("Grado di affidabilità: %.3f\n" % max_score)
print("%d/%d -> Questa etichetta è stata predetta %d volte su %d." 
      % (occorrenze[label_predict], newRECORDS_NUMBER, occorrenze[label_predict], newRECORDS_NUMBER))

Etichetta predetta: Pages 



Grado di affidabilità: 0.759

91/91 -> Questa etichetta è stata predetta 91 volte su 91.


# Label : Reviews

In [203]:
url='http://localhost:8123/?query=SELECT top 100 iiliriiriliiriliiiirlrrl  FROM extracted_data.57632537416d4cd19f3965026ab51caf'

try:
    r = requests.get(url)
except ConnectionError:
        print("Il collegamento al database non è attivo.")

        
#L'utf-8-sig è una variante Python di UTF-8 che ci permette di eliminare, se presenti, eventuali carattere UTF-8 BOM.
values= r.content.decode('utf-8-sig')


#Controllo i parametri inseriti dall'utente
if (values[6:8]=='60'):
    print("ATTENZIONE! La tabella indicata non esiste.")
elif (values[6:8]=='47'):
    print("ATTENZIONE! La colonna indicata non esiste.")
else:
    print("I dati sono stati caricati correttamente.")

I dati sono stati caricati correttamente.


In [204]:
values

'\n\n\n\nLate on a hot summer night at the tail end of 1965, Charlie Bucktin, a precocious and bookish boy of thirteen, is startled by a knock on his window. His visitor is Jasper Jones. Rebellious, mixed-race and solitary, Jasper is a distant figure of danger and intrigue for Charlie.\n\nAmphoto\\\'s bestselling title of all time, now updated to reflect the quickly changing world of digital photography. Understanding Exposure has demystified the often-complex concepts of exposure for hundreds of thousands of amateur photographers. Now refreshed with current technologies and terms and an all-new chapter, Bryan Peterson continues in "bringing exposure to the masses," teaching today\\\'s a...\n\nA guide to luxury lifestyle design. It teaches: how the author went from $40,000 dollars per year and 80 hours per week to $40,000 per month and 4 hours per week; how to outsource your life to overseas virtual assistants for $5 per hour and do whatever you want; how blue-chip escape artists trave

In [205]:
values = values.replace("\\"," ")

In [206]:
values

'\n\n\n\nLate on a hot summer night at the tail end of 1965, Charlie Bucktin, a precocious and bookish boy of thirteen, is startled by a knock on his window. His visitor is Jasper Jones. Rebellious, mixed-race and solitary, Jasper is a distant figure of danger and intrigue for Charlie.\n\nAmphoto \'s bestselling title of all time, now updated to reflect the quickly changing world of digital photography. Understanding Exposure has demystified the often-complex concepts of exposure for hundreds of thousands of amateur photographers. Now refreshed with current technologies and terms and an all-new chapter, Bryan Peterson continues in "bringing exposure to the masses," teaching today \'s a...\n\nA guide to luxury lifestyle design. It teaches: how the author went from $40,000 dollars per year and 80 hours per week to $40,000 per month and 4 hours per week; how to outsource your life to overseas virtual assistants for $5 per hour and do whatever you want; how blue-chip escape artists travel 

In [207]:
values = values.split("\n")
values

['',
 '',
 '',
 '',
 'Late on a hot summer night at the tail end of 1965, Charlie Bucktin, a precocious and bookish boy of thirteen, is startled by a knock on his window. His visitor is Jasper Jones. Rebellious, mixed-race and solitary, Jasper is a distant figure of danger and intrigue for Charlie.',
 '',
 'Amphoto \'s bestselling title of all time, now updated to reflect the quickly changing world of digital photography. Understanding Exposure has demystified the often-complex concepts of exposure for hundreds of thousands of amateur photographers. Now refreshed with current technologies and terms and an all-new chapter, Bryan Peterson continues in "bringing exposure to the masses," teaching today \'s a...',
 '',
 'A guide to luxury lifestyle design. It teaches: how the author went from $40,000 dollars per year and 80 hours per week to $40,000 per month and 4 hours per week; how to outsource your life to overseas virtual assistants for $5 per hour and do whatever you want; how blue-ch

In [208]:
values = list(filter(lambda x : x != '', values))

In [209]:
values

['Late on a hot summer night at the tail end of 1965, Charlie Bucktin, a precocious and bookish boy of thirteen, is startled by a knock on his window. His visitor is Jasper Jones. Rebellious, mixed-race and solitary, Jasper is a distant figure of danger and intrigue for Charlie.',
 'Amphoto \'s bestselling title of all time, now updated to reflect the quickly changing world of digital photography. Understanding Exposure has demystified the often-complex concepts of exposure for hundreds of thousands of amateur photographers. Now refreshed with current technologies and terms and an all-new chapter, Bryan Peterson continues in "bringing exposure to the masses," teaching today \'s a...',
 'A guide to luxury lifestyle design. It teaches: how the author went from $40,000 dollars per year and 80 hours per week to $40,000 per month and 4 hours per week; how to outsource your life to overseas virtual assistants for $5 per hour and do whatever you want; how blue-chip escape artists travel the w

In [210]:
newRECORDS_NUMBER = len(values)
len(values)

52

In [216]:
#Struttura: dictionary[label] -> [89%, 93%, 97%, 32%, 99% , ...]
dictionary={}

#Sruttura ausiliaria: permette di stabilire il numero di volte in cui ciascuna etichetta è stata predetta
occorrenze={}


#Crea una lista vuota per ciascuna label 
for label in labels:
    dictionary[label]=[]
    occorrenze[label]=0
    

classifier = pipeline("zero-shot-classification")
    
labels_number= len(labels)

#Vengono effettuate le previsioni attraverso il modello.
#Le previsioni vengono memorizzate nel dizionario.
for i in range(newRECORDS_NUMBER):
    text = values[i]
    result=classifier(text, labels)
    occorrenze[result['labels'][0]]+=1
    for y in range(labels_number):
        label= result['labels'][y]
        score= result['scores'][y]
        dictionary[label].append(score)    
        
        
risultati={}
for label in labels:
    risultati[label]= sum(dictionary[label])/newRECORDS_NUMBER

max_score= 0

for label, score in risultati.items():  
    if score > max_score:
        label_predict=label
        max_score=score

print("Etichetta predetta: %s \n\n\n" % label_predict)
print("Grado di affidabilità: %.3f\n" % max_score)
print("%d/%d -> Questa etichetta è stata predetta %d volte su %d." 
      % (occorrenze[label_predict], newRECORDS_NUMBER, occorrenze[label_predict], newRECORDS_NUMBER))

Etichetta predetta: Description 



Grado di affidabilità: 0.295

25/52 -> Questa etichetta è stata predetta 25 volte su 52.
